##  <font color=Green>사용하는 함수</font>


###  <font color=blue>1.SetBehaviorClient(비해비어 숫자, 로봇이름)</font>

###### 예시
- SetBehaviorClient(1,"robot_0")
###### 비해비어 인덱스 (19년3월2일 기준)
- 0: ready (아무것도안함)
- 1: lockon search behavior (움직이다가 적발견하면 락온하고 쏨. 목표로 x,y좌표받음)
- 2: manual speed control (수동으로 움직이면서 락온함. 목표로 x,y속도랑 바라보는 방향 theta 받음. 지도 절대좌표기준임)
- 3: hold rotate behavior (제자리에서 빙글빙글 돌다가 적발견하면 락온함. 목표로 각속도 받게 할 예정. 일단은 2로 돌음)
- 4: hold sentry behavior (특정 방향 좌우로 45도 두리번거리다가 적발견하면 락온함. 목표로 바라볼 방향 받음)
- 5: reload behavior (재장전함. 목표로 장전 위치 정해줘야함. ex np.array([4,4.5,-1.5707,0,0,0,0]) )
- 6: get buff behavior (미구현)
- 7: backboot behavior (처음시작 위치로 가서 정렬함. 목표로 처음 시작 위치 정해줘야함 np.array([1,1,0,0,0,0,0]))
- 나머진 아직안됨


###  <font color=blue>2. SetGoalClient(목표, 로봇이름)</font>
###### 예시
- goal = np.array([5,3,0,0,0,0,0])
- SetGoalClient(goal,"robot_0")
###### 기타
- 넘파이 7차원벡터를 goal로 받음. behavior마다 쓰는 goal이 다름
- 일반적으로 [x,y,yaw,vx,vy,v_th,etc] 혹은 [vx,vy,v_th,ax,ay,a_th,etc]로 생각하면됨
- lockon search behavior 같은 navigation 계열은 x,y좌표만 받음. 따라서 0번, 1번 element만 채워주면됨
- manual speed behavior같은 manual control게열은 x속도 y속도, 각도관련 받음 따라서 0 1 2 element 사용
- hold 게열은 theta만 받음 고로 2번 element만 채워주면됨
- 등등

###  <font color=blue>3. GetInfoClient(로봇이름,확인)</font>
###### 설명
- 로봇의 상태(블랙보드)를 딕셔너리로 받아옴. 확인은 0은 false, 1은 true로서 일단은 적에게 맞았을 때 is hit이 true가 되는데 이걸 다시 false로 바꿔줄것이냐 아니냐를 의미함. 가령 단순히 로봇의 상태만 보기 위해서는 0을 설정하고면 되고, 실제 의사결정을 하는 디시젼노드가 이 정보를 받을 때는 적에게 맞았다는것을 확인했기 때문에 is hit을 false로 바꿔줘야 하므로 확인을 1로 설정함. 기본값은 1임 잘 모르겠으면 1로하면됨
###### 예시
- GetInfoClient("robot_0",1)


##  <font color=Green>실제 예시</font>

In [17]:
import matplotlib

matplotlib.use('TkAgg')

import copy
import pylab
import random
import numpy as np
from collections import deque
#import tensorflow as tf
import keras.backend.tensorflow_backend as K
#config = tf.ConfigProto()
#config.gpu_options.allow_growth=True
#session=tf.Session(config=config)
import keras
#keras.backend.set_session(session)
from keras.layers import Dense, Conv1D, Flatten
from keras.optimizers import Adam
from keras.models import Sequential
import time

EPISODES = 100

Using TensorFlow backend.


In [100]:
%pwd

u'/home/kang/catkin_ws/src/icra_roboin_decision/scripts'

In [39]:
import rospy
import roslib
import numpy as np
from icra_roboin_decision_modules.roboin_behavior_service_module import SetBehaviorClient, SetGoalClient, GetInfoClient
rospy.init_node('decision_test')

In [18]:
class DQN_Agent:
    def __init__(self, state_size):
        self.load_model = False
        self.action_space = ['u', 'd', 'l', 'r','ul','ur','dl','dr','enemy','back']
        self.action_size = len(self.action_space)
        self.state_size = state_size

        # Hyper_Parameters
        self.discount_factor = 0.99
        self.learning_rate = 0.0001
        self.epsilon = 1
        self.epsilon_decay = .99999
        self.epsilon_min = 0.1
        self.batch_size = 32
        self.train_start = 2500

        # Model & Target Model
        self.model = self.build_model()
        self.target_model = self.build_model()
        print(self.model.summary())
        self.memory = deque(maxlen=100000)

        if self.load_model:
            self.epsilon = 0.05
            self.model.load_weights('./save_model/DQN_Agent.h5')

    # Network
    def build_model(self):
        with K.tf.device('/cpu:0'):
            model = Sequential()
            model.add(Conv1D(filters = 10, kernel_size = 1, input_shape = (self.state_size, 3),
                             kernel_initializer = 'he_uniform', activation = 'tanh'))
            model.add(Dense(30, activation = 'tanh', kernel_initializer = 'he_uniform'))
            model.add(Dense(30, activation = 'tanh', kernel_initializer = 'he_uniform'))
            model.add(Flatten())
            model.add(Dense(self.action_size, activation = 'linear'))
            model.summary()
            model.compile(loss = 'mse', optimizer = Adam(lr = self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Choose Action by e-greedy
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            # Random Action
            return random.randrange(self.action_size)
        else:
            # Action by model
            # state = np.float32(state)
            q_values = self.model.predict(state)
            #print("Q_Values: ", q_values)
            return np.argmax(q_values[0])

    # <state, action, reward, next_state> in Replay Memory
    def append_sample(self, state, action, reward, next_state, done):
        # state = np.reshape(state, [1, -1])
        # next_state = np.reshape(next_state, [1, -1])
        self.memory.append((state, action, reward, next_state, done))

    # Random Sampling in Replay Memory, Model Training
    def train_model(self):
        with K.tf.device('/cpu:0'):
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

            # Random Sampling in memory
            mini_batch = random.sample(self.memory, self.batch_size)
            #states = [[0 for x in range(self.state_size)] for y in range(self.batch_size)]
            states = np.zeros((self.batch_size,1 ,self.state_size,3))
            #next_states = [[0 for x in range(self.state_size)] for y in range(self.batch_size)]
            next_states = np.zeros((self.batch_size, 1, self.state_size,3))
            actions, rewards, dones = [], [], []

            for i in range(self.batch_size):
                states[i] = mini_batch[i][0]
                next_states[i] = mini_batch[i][3]
                actions.append(mini_batch[i][1])
                rewards.append(mini_batch[i][2])
                dones.append(mini_batch[i][4])

            target = np.zeros((self.batch_size,1,4))
            target_val = np.zeros((self.batch_size,1,4))
            for i in range(len(states)):
                target[i] = self.model.predict(states[i])
                target_val[i] = self.target_model.predict(next_states[i])
            for i in range(self.batch_size):
                if dones[i]:
                    target[i][0][actions[i]] = rewards[i]
                else:
                    target[i][0][actions[i]] = rewards[i] + self.discount_factor * np.amax(target_val[i])

            states = np.squeeze(states)
            target = np.squeeze(target)

            self.model.fit(states, target, batch_size=self.batch_size,
                           epochs=1, verbose=0)

In [20]:
rewards ={}
temp_info = GetInfoClient("robot_0",1)
rewards['enemy_detect'] = temp_info['how_many_enemies_detected'] * 1
rewards['is_hit'] = temp_info['is_hit'] * -1
# rewards['my_health'] = temp_info['my_health'] * 0.001

In [98]:
##reset
def robo_reset():
    behav_=7
    SetBehaviorClient(behav_,"robot_0")
    SetBehaviorClient(behav_,"robot_1")
    goal_ = np.array([1,1,0,0,0,0,0])
    SetGoalClient(goal_,"robot_0")
    SetGoalClient(goal_,"robot_1")
    reset = True
    while reset:
        check_pos0 = GetInfoClient("robot_0",1)["my_pose"]["pose"]["position"]
        check_pos1 = GetInfoClient("robot_1",1)["my_pose"]["pose"]["position"]
        pos0_norm = pow(pow(check_pos0['x'],2)+pow(check_pos0['y'],2),0.5)
        pos1_norm = pow(pow(check_pos1['x'],2)+pow(check_pos1['y'],2),0.5)
        print pos0_norm, pos1_norm
        if (pos0_norm<pow(2.1, 0.5)) and (pos1_norm<pow(2.1, 0.5)):
            reset = False

In [109]:
robo_reset()

1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.36921319773
1.45706279736 1.3692

KeyboardInterrupt: 

In [103]:
def move(robot, action):
    action_space = ['u', 'd', 'l', 'r','ul','ur','dl','dr','enemy','back']
    movement = action_space[action]
    
    move_const = 0.5
    SetBehaviorClient(1, robot)
    temp_info = GetInfoClient(robot,1)
    my_pos = temp_info["my_pose"]["pose"]["position"]
    enemy_pos = temp_info["enemy_pose1"]["pose"]["position"]
    goal = np.array([my_pos['x'], my_pos['y'], 0, 0, 0, 0, 0])
    root2 = pow(2, 0.5)
    if movement=='u':
        goal += np.array([0,1,0,0,0,0,0]) * move_const
    elif movement=='d':
        goal += np.array([0,-1,0,0,0,0,0]) * move_const
    elif movement=='l':
        goal += np.array([-1,0,0,0,0,0,0]) * move_const
    elif movement=='r':
        goal += np.array([1,0,0,0,0,0,0]) * move_const
    elif movement=='ul':
        goal += np.array([-root2,root2,0,0,0,0,0]) * move_const
    elif movement=='ur':
        goal += np.array([root2,root2,0,0,0,0,0]) * move_const
    elif movement=='dl':
        goal += np.array([-root2,-root2,0,0,0,0,0]) * move_const
    elif movement=='dr':
        goal += np.array([root2,-root2,0,0,0,0,0]) * move_const
    elif movement=='enemy':
        goal = np.array([enemy_pos['x'],enemy_pos['y'],0,0,0,0,0])
    elif movement=='back':
        goal = np.array([1,1,0,0,0,0,0])
    SetGoalClient(goal, robot)

In [110]:
def get_info(robot):
    temp_info = GetInfoClient(robot,1)
    rewards['enemy_detect'] = temp_info['how_many_enemies_detected'] * 1
    rewards['is_hit'] = temp_info['is_hit'] * -1
    enemy1_position = temp_info['enemy_pose1']['pose']['position']
    enemy1_position = [enemy1_position['x'],enemy1_position['y']]
    my_position = temp_info['my_pose']['pose']['position']
    my_position = [my_position['x'],my_position['y']]
    #reward is a number
    reward = rewards['enemy_detect']+rewards['is_hit']
    state = my_position+enemy1_position+[reward]
    return state,reward

state_size = 5 # len(state)
agent = DQN_Agent(state_size)
episode = 10
rewards = {}
scores, episodes = [], []
# get 1st state
state0, _ = get_info("robot_0")
state1, _ = get_info("robot_1")
#robo_reset()
for e in range(1,episode+1):
    done = False
    score = 0
    step = 0
    global_step = 0
    step_term = 0
    while not done:
        step += 1
        global_step += 1
        start_time = time.time()
        onestep_start = time.time()
        
        action0 = agent.get_action(state0)
        action1 = agent.get_action(state1)
        
        time.sleep(0.05)
        move("robot_0", action0)
        move("robot_1", action1)
        next_state0, reward0 = get_info("robot_0")
        next_state1, reward1 = get_info("robot_1")
        reward0 += -0.01
        reward1 += -0.01
        if step == 200:
            reward0 -= 10
            reward1 -= 10
            done = True
        agent.append_sample(state0,action0,reward0,next_state0,done)
        agent.append_sample(state1,action1,reward1,next_state1,done)

            
        score += reward0
        score += reward1
        
        state0 = copy.deepcopy(next_state0)
        state1 = copy.deepcopy(next_state1)
        
        if done:
            agent.update_target_model()
            scores.append(score)
            episodes.append(e)
            end_time = time.time()
            print ("episode : ", e, "step : ", step, "Replay_Memory : ", len(agent.memory),
                   'global_step : ', global_step, 'epsilon : ', agent.epsilon)
            robo_reset()
                
        onestep_end=time.time()
        step_time = onestep_end - onestep_start
        step_term += step_time
        if step % 10 == 0:
            print 'Step : ', step, 'time : ', int(step_term*1000), 'ms', 'score : ', score
            step_term=0
        if (global_step % 100 == 0) or (e == EPISODES):
            pylab.plot(episodes, scores, 'olive')
            pylab.savefig("./save_graph/DQN_Agent.png")
            
    # Save model
    if e % 20 == 0:
        agent.model.save_weights("./save_model/DQN_Agent.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 5, 10)             40        
_________________________________________________________________
dense_49 (Dense)             (None, 5, 30)             330       
_________________________________________________________________
dense_50 (Dense)             (None, 5, 30)             930       
_________________________________________________________________
flatten_17 (Flatten)         (None, 150)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 10)                1510      
Total params: 2,810
Trainable params: 2,810
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1

KeyboardInterrupt: 

In [ ]:
SetBehaviorClient(1,"robot_0")
goal_ = np.array([5,3,0,0,0,0,0])
SetGoalClient(goal_,"robot_0")

In [ ]:
GetInfoClient("robot_1",1)["my_health"]

In [6]:
GetInfoClient("robot_0",1)

{'ammo': 50,
 'buff_left': {'nsecs': 0, 'secs': 0},
 'buff_zone_cooltime': {'nsecs': 0, 'secs': 0},
 'current_behavior_process': 2,
 'current_behavior_style': 1,
 'enemy_pose1': {'header': {'frame_id': '',
   'seq': 0,
   'stamp': {'nsecs': 0, 'secs': 0}},
  'pose': {'orientation': {'w': 0.0, 'x': 0.0, 'y': 0.0, 'z': 0.0},
   'position': {'x': 0.0, 'y': 0.0, 'z': 0.0}}},
 'enemy_pose2': {'header': {'frame_id': '',
   'seq': 0,
   'stamp': {'nsecs': 0, 'secs': 0}},
  'pose': {'orientation': {'w': 0.0, 'x': 0.0, 'y': 0.0, 'z': 0.0},
   'position': {'x': 0.0, 'y': 0.0, 'z': 0.0}}},
 'enemy_priority': 3,
 'game_start_time': {'nsecs': 400000000, 'secs': 4},
 'game_state': 2,
 'goal': {'etc': 0.0,
  'header': {'frame_id': 'robot_0/map',
   'seq': 0,
   'stamp': {'nsecs': 200000000, 'secs': 193}},
  'x': 2.061732172649977,
  'xa': 0.0,
  'y': 0.9734800516102516,
  'ya': 0.0,
  'yaw': 0.0,
  'yawa': 0.0},
 'has_buff': False,
 'how_many_enemies_detected': 0,
 'is_enemy_1_detected': False,
 'is_

In [ ]:
# reset game
# when referee says game is finished


behav_=7
SetBehaviorClient(behav_,"robot_0")
goal_ = np.array([1,1,0,0,0,0,0])
SetGoalClient(goal_,"robot_0")
behav_=7
SetBehaviorClient(behav_,"robot_1")
goal_ = np.array([1,1,0,0,0,0,0])
SetGoalClient(goal_,"robot_1")

##  <font color=Green>시나리오</font>

In [ ]:
info_0 = GetInfoClient("robot_0",1)
info_1 = GetInfoClient("robot_1",1)
print "robot_0 health: " + str(info_0["my_health"])
print "robot_1 health: " + str(info_1["my_health"])
print "robot_0 ammo: " + str(info_0["ammo"])
print "robot_1 ammo: " + str(info_1["ammo"])
print "robot_0 game_state: " + str(info_0["game_state"])
print "robot_1 game_state: " + str(info_1["game_state"])
print "robot_0 behavior: " + str(info_0["current_behavior_style"])
print "robot_1 behavior: " + str(info_1["current_behavior_style"])
print "robot_0 process: " + str(info_0["current_behavior_process"])
print "robot_1 process: " + str(info_1["current_behavior_process"])

In [ ]:
# SCENARIO

# blue and red meets at 5,3 and engage for 9 sec

# red robot flanks
# blue robot turns to the direction of the armor
# engage again

# red goes to reload and after 1 sec blue also reloads

# after 10 sec, again meet at 1,1 and fight until one dies

import time

behav_=1
SetBehaviorClient(behav_,"robot_0")
goal_ = np.array([5,3,0,0,0,0,0])
SetGoalClient(goal_,"robot_0")
behav_=1
SetBehaviorClient(behav_,"robot_1")
goal_ = np.array([3,2,0,0,0,0,0])
SetGoalClient(goal_,"robot_1")

time.sleep(9)

behav_=1
SetBehaviorClient(behav_,"robot_1")
goal_ = np.array([5,3,0,0,0,0,0])
SetGoalClient(goal_,"robot_1")
behav_=4
SetBehaviorClient(behav_,"robot_0")
goal_ = np.array([5,3,1,0,0,0,0])
SetGoalClient(goal_,"robot_0")

time.sleep(9)

behav_=1
SetBehaviorClient(behav_,"robot_0")
goal_ = np.array([5,3,0,0,0,0,0])
SetGoalClient(goal_,"robot_0")

time.sleep(0.8)

behav_=1
SetBehaviorClient(behav_,"robot_1")
goal_ = np.array([3,3,0,0,0,0,0])
SetGoalClient(goal_,"robot_1")

time.sleep(3)

behav_=5
SetBehaviorClient(behav_,"robot_1")
goal_ = np.array([4,4.5,-1.5707,0,0,0,0])
SetGoalClient(goal_,"robot_1")

time.sleep(1)

behav_=5
SetBehaviorClient(behav_,"robot_0")
goal_ = np.array([4,4.5,-1.5707,0,0,0,0])
SetGoalClient(goal_,"robot_0")

time.sleep(10)

behav_=1
SetBehaviorClient(behav_,"robot_0")
goal_ = np.array([1,1,0,0,0,0,0])
SetGoalClient(goal_,"robot_0")
behav_=1
SetBehaviorClient(behav_,"robot_1")
goal_ = np.array([7,4,0,0,0,0,0])
SetGoalClient(goal_,"robot_1")